In [1]:
import json
with open('group_compression_tests.json', 'r', encoding='utf8') as jfile:
    data = json.load(jfile)

In [6]:
c_map = {'infer': 'uncompressed', 'none': 'uncompressed'}
file_dict = {}
for filetype in data.keys():
    file_dict[filetype] = {}
    for compression in data[filetype].keys():
        new_c_key = c_map.get(compression, compression)
        file_dict[filetype][new_c_key] = {**data[filetype][compression]}


In [7]:
with open('fixed_group_compression_tests.json', 'w', encoding='utf8') as jfile:
    json.dump(file_dict, jfile, indent=4)

In [8]:
with open('fixed_group_compression_tests.json', 'r', encoding='utf8') as jfile:
    tests = json.load(jfile)

In [45]:
ranking_dict = {}
award_points = lambda sorted_list, weight: list(zip(sorted_list, [i * weight for i, _ in enumerate(sorted_list, start=1)]))
filesize_weight = 2
readtime_weight = 1.5
writetime_weight = 1

for filetype in tests.keys():
    read_times = [[compression, results['mean_read_time']] for compression, results in tests[filetype].items() if compression != 'uncompressed']
    write_times = [[compression, results['mean_write_time']] for compression, results in tests[filetype].items() if compression != 'uncompressed']
    filesizes = [[compression, results['mean_filesize_in_GB']] for compression, results in tests[filetype].items() if compression != 'uncompressed']


    # Lowest time fi
    sorted_read_times = sorted(read_times, key=lambda x: x[1], reverse=True)
    sorted_write_times = sorted(write_times, key=lambda x: x[1], reverse=True)
    sorted_filesizes = sorted(filesizes, key=lambda x: x[1], reverse=True)
    ranking_dict[filetype] = {
        'readtimes': award_points(sorted_read_times, readtime_weight),
        'writetimes': award_points(sorted_write_times, writetime_weight),
        'filesizes': award_points(sorted_filesizes, filesize_weight)
    }

In [46]:
from collections import defaultdict
standings = {}
for filetype in ranking_dict.keys():
    standings[filetype] = defaultdict(list)
    for challenge in ranking_dict[filetype].keys():
        for result in ranking_dict[filetype][challenge]:
            [compression, _], points = result
            standings[filetype][compression].append(points)
    
    
    for compression, points in standings[filetype].items():
        print(filetype,  compression, sum(points))
        #compression, measure, points = ranking_dict[filetype][challenge]
print(standings)

parquet gzip 9.5
parquet brotli 12.0
parquet zstd 17.5
parquet snappy 15.0
parquet lz4 13.5
orc zlib 8.5
orc zstd 13.0
orc snappy 11.5
orc lz4 12.0
feather zstd 6.5
feather lz4 7.0
csv bz2 11.5
csv xz 14.0
csv gzip 13.5
csv zip 12.0
csv zstd 16.5
{'parquet': defaultdict(<class 'list'>, {'gzip': [1.5, 2, 6], 'brotli': [3.0, 1, 8], 'zstd': [4.5, 3, 10], 'snappy': [6.0, 5, 4], 'lz4': [7.5, 4, 2]}), 'orc': defaultdict(<class 'list'>, {'zlib': [1.5, 1, 6], 'zstd': [3.0, 2, 8], 'snappy': [4.5, 3, 4], 'lz4': [6.0, 4, 2]}), 'feather': defaultdict(<class 'list'>, {'zstd': [1.5, 1, 4], 'lz4': [3.0, 2, 2]}), 'csv': defaultdict(<class 'list'>, {'bz2': [1.5, 2, 8], 'xz': [3.0, 1, 10], 'gzip': [4.5, 3, 6], 'zip': [6.0, 4, 2], 'zstd': [7.5, 5, 4]})}
